In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-prices-dataset/Housing.csv


Importing the dataset and dividing to independent and dependent variables

In [2]:
dt = pd.read_csv('../input/housing-prices-dataset/Housing.csv')
dt.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [4]:
x = dt.iloc[:,1:].values
y = dt.iloc[:, 0].values

In [5]:
dt.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

Encoding furnishingstatus column using OneHotEncoder

In [6]:
x[0]

array([7420, 4, 2, 3, 'yes', 'no', 'no', 'no', 'yes', 2, 'yes',
       'furnished'], dtype=object)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))
x[0]

array([1.0, 0.0, 0.0, 7420, 4, 2, 3, 'yes', 'no', 'no', 'no', 'yes', 2,
       'yes'], dtype=object)

LabelEncoder for yes, no

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,7] = le.fit_transform(x[:,7].astype(str))
x[:,8] = le.fit_transform(x[:,8].astype(str))
x[:,9] = le.fit_transform(x[:,9].astype(str))
x[:,10] = le.fit_transform(x[:,10].astype(str))
x[:,11] = le.fit_transform(x[:,11].astype(str))
x[:,13] = le.fit_transform(x[:,13].astype(str))
x[0:3]

array([[1.0, 0.0, 0.0, 7420, 4, 2, 3, 1, 0, 0, 0, 1, 2, 1],
       [1.0, 0.0, 0.0, 8960, 4, 4, 4, 1, 0, 0, 0, 1, 3, 0],
       [0.0, 1.0, 0.0, 9960, 3, 2, 2, 1, 0, 1, 0, 0, 2, 1]], dtype=object)

Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state = 1)

Feature Scaling, normalizing range of values

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train[:, 3:4] = sc.fit_transform(x_train[:, 3:4])
x_test[:, 3:4] = sc.transform(x_test[:, 3:4])
x_train[0:4]

array([[0.0, 1.0, 0.0, 2.2176003337956427, 3, 1, 1, 1, 0, 0, 0, 0, 1, 1],
       [0.0, 1.0, 0.0, 0.38142960100838186, 3, 1, 3, 1, 1, 0, 0, 1, 0, 1],
       [1.0, 0.0, 0.0, 3.0196312535379315, 4, 2, 2, 1, 0, 0, 0, 0, 2, 1],
       [0.0, 0.0, 1.0, 0.2845340478269961, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0]],
      dtype=object)